Password match - get user ID

In [7]:
from services.user_dao import password_match, get_user_id

In [8]:
admin = "admin"
password = "admin"

In [9]:
if not password_match(admin, password):
    raise Exception("Login failed")

user_id = get_user_id(admin)
print(f"Login successful – User ID: {user_id}")

Login successful – User ID: 1


Retrieve all portfolios and transaction history related to this ID

In [10]:
from services.portfolio_dao import get_portfolios_by_user

In [11]:
portfolios = get_portfolios_by_user(user_id)
portfolio_id_to_name = {p.id: p.name for p in portfolios}

In [17]:
from services.transaction_dao import get_transactions_for_portfolio

In [18]:
portfolio_transactions = {
    p.id: get_transactions_for_portfolio(p.id)
    for p in portfolios
}

In [19]:
frontend_transactions = []
for pid, txs in portfolio_transactions.items():
    portfolio_name = portfolio_id_to_name.get(pid, "Unknown Portfolio")
    for tx in txs:
        frontend_transactions.append({
            "id": tx["id"],
            "portfolio": portfolio_name,
            "ticker": tx["ticker"],
            "type": tx["type"].title(),
            "quantity": tx["quantity"],
            "price": tx["price"],
            "dateTime": tx["executed_at"].strftime("%Y-%m-%d %H:%M:%S")
        })

In [20]:
frontend_transactions

[{'id': 1,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'AAPL',
  'type': 'Buy',
  'quantity': 10,
  'price': 150.0,
  'dateTime': '2025-04-15 18:34:45'},
 {'id': 2,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'GOOGL',
  'type': 'Buy',
  'quantity': 5,
  'price': 2800.0,
  'dateTime': '2025-04-15 18:34:45'},
 {'id': 5,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'MSFT',
  'type': 'Buy',
  'quantity': 10,
  'price': 100.0,
  'dateTime': '2025-04-22 00:22:51'},
 {'id': 6,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'AAPL',
  'type': 'Sell',
  'quantity': 5,
  'price': 150.0,
  'dateTime': '2025-04-22 00:28:24'},
 {'id': 8,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'APPL',
  'type': 'Buy',
  'quantity': 10,
  'price': 100.0,
  'dateTime': '2025-04-22 00:31:22'},
 {'id': 9,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'AAPL',
  'type': 'Buy',
  'quantity': 10,
  'price': 150.0,
  'dateTime': '2025-04-30 02:32:21'},
 {'id': 10,
  'portfolio': 'Retir

Fetch latest market data related to our portfolio stocks

In [1]:
from services.market_service import fetch_market_data

In [11]:
stock_lookup = {s['ticker']: s for s in fetch_market_data()}

Retreive all Stocks (positions) related to our portfolios

In [ ]:
from services.market_service import fetch_market_data
from services.position_dao import get_positions_by_portfolio

# One source of truth
market_data = fetch_market_data()
stock_lookup = {s["ticker"]: s for s in market_data}

def calculate_position_metrics(quantity, avg_price, current_price):
    market_value = quantity * current_price
    cost_basis = quantity * avg_price
    profit_loss = market_value - cost_basis
    profit_loss_percent = ((current_price - avg_price) / avg_price) * 100 if avg_price != 0 else 0
    return market_value, cost_basis, profit_loss, profit_loss_percent

portfolio_summary = []

for portfolio in portfolios:
    positions = get_positions_by_portfolio(portfolio.id)
    positions_list = []

    for p in positions:
        stock_info = stock_lookup.get(p.ticker, {})

        current_price = stock_info.get("price", 0)
        mv, cb, pnl, pnl_pct = calculate_position_metrics(p.quantity, float(p.average_price), current_price)

        positions_list.append({
            "id": p.id,
            "ticker": p.ticker,
            "name": stock_info.get("name", p.ticker),
            "sector": stock_info.get("sector", "Unknown"),
            "quantity": p.quantity,
            "average_price": float(p.average_price),
            "created_at": p.created_at.isoformat(),
            "updated_at": p.updated_at.isoformat(),
            "current_price": current_price,
            "market_value": round(mv, 2),
            "cost_basis": round(cb, 2),
            "profit_loss": round(pnl, 2),
            "profit_loss_percent": round(pnl_pct, 2),
            "percent_change": stock_info.get("change", 0),
            "market_cap": stock_info.get("marketCap", "N/A")
        })

    portfolio_summary.append({
        "portfolio_id": portfolio.id,
        "portfolio_name": portfolio.name,
        "strategy": portfolio.strategy.value,
        "positions": positions_list
    })

Buy a stock and save the transaction

In [16]:
from services.transaction_dao import add_transaction
from services.position_dao import buy_stock
from models.transactions import TransactionType

In [17]:
portfolio_id = 1
ticker = "META"
quantity = 11
price = 175.00

In [18]:
message = buy_stock(portfolio_id, ticker, quantity, price)
print(message)
add_transaction(portfolio_id, ticker, quantity, price, TransactionType.buy)

Successfully bought 11 of META stock at $175.00


Sell a stock and save the transaction

In [ ]:
from services.position_dao import sell_stock
from services.transaction_dao import add_transaction
from models.transactions import TransactionType

In [2]:
portfolio_id = 1
ticker = "META"
quantity = 11
price = 175.00

In [ ]:
sell_stock(portfolio_id, ticker, quantity, price)
add_transaction(portfolio_id, ticker, quantity, price, TransactionType.sell)

'Successfully sold 11 of META stock at $175.00'

In [12]:
from services.market_service import fetch_market_data
from services.position_dao import get_positions_by_portfolio

# One source of truth
market_data = fetch_market_data()
stock_lookup = {s["ticker"]: s for s in market_data}

def calculate_position_metrics(quantity, avg_price, current_price):
    market_value = quantity * current_price
    cost_basis = quantity * avg_price
    profit_loss = market_value - cost_basis
    profit_loss_percent = ((current_price - avg_price) / avg_price) * 100 if avg_price != 0 else 0
    return market_value, cost_basis, profit_loss, profit_loss_percent

portfolio_summary = []

for portfolio in portfolios:
    positions = get_positions_by_portfolio(portfolio.id)
    positions_list = []

    for p in positions:
        stock_info = stock_lookup.get(p.ticker, {})

        current_price = stock_info.get("price", 0)
        mv, cb, pnl, pnl_pct = calculate_position_metrics(p.quantity, float(p.average_price), current_price)

        positions_list.append({
            "id": p.id,
            "ticker": p.ticker,
            "name": stock_info.get("name", p.ticker),
            "sector": stock_info.get("sector", "Unknown"),
            "quantity": p.quantity,
            "average_price": float(p.average_price),
            "created_at": p.created_at.isoformat(),
            "updated_at": p.updated_at.isoformat(),
            "current_price": current_price,
            "market_value": round(mv, 2),
            "cost_basis": round(cb, 2),
            "profit_loss": round(pnl, 2),
            "profit_loss_percent": round(pnl_pct, 2),
            "percent_change": stock_info.get("change", 0),
            "market_cap": stock_info.get("marketCap", "N/A")
        })

    portfolio_summary.append({
        "portfolio_id": portfolio.id,
        "portfolio_name": portfolio.name,
        "strategy": portfolio.strategy.value,
        "positions": positions_list
    })

In [13]:
portfolio_summary

[{'portfolio_id': 1,
  'portfolio_name': 'Retirement Portfolio',
  'strategy': 'long_term',
  'positions': [{'id': 1,
    'ticker': 'AAPL',
    'name': 'Apple Inc.',
    'sector': 'Technology',
    'quantity': 84,
    'average_price': 160.72,
    'created_at': '2025-04-15T18:34:24',
    'updated_at': '2025-04-30T03:11:42',
    'current_price': 212.5,
    'market_value': 17850.0,
    'cost_basis': 13500.48,
    'profit_loss': 4349.52,
    'profit_loss_percent': 32.22,
    'percent_change': 1.29,
    'market_cap': '3.19T'},
   {'id': 2,
    'ticker': 'GOOGL',
    'name': 'Alphabet Inc.',
    'sector': 'Communication Services',
    'quantity': 5,
    'average_price': 2800.0,
    'created_at': '2025-04-15T18:34:24',
    'updated_at': '2025-04-15T18:34:24',
    'current_price': 158.8,
    'market_value': 794.0,
    'cost_basis': 14000.0,
    'profit_loss': -13206.0,
    'profit_loss_percent': -94.33,
    'percent_change': -1.36,
    'market_cap': '1.95T'},
   {'id': 7,
    'ticker': 'META',